In [1]:
import numpy as np
import time
import tensorflow as tf

In [2]:
from eventcnn.datasources.davis import DavisDataset
from eventcnn.datasources.eventsource import EventSource, CachedDenseEventSource

In [3]:
source = CachedDenseEventSource("data/dense")

In [9]:
N_EVENT_SLICES = source.event_slices
BATCH_SIZE = 5
N_EVENT_ROWS = source.rows
N_EVENT_COLS = source.cols

In [5]:
# Copied from the tensorflow tutorial in class
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def init_bias(shape):
    return tf.Variable(tf.zeros(shape))

In [6]:
def n_output_channels(layer):
    return layer.get_shape()[-1].value

def n_outputs(layer):
    out = layer.get_shape()[1].value
    for d in layer.get_shape()[2:]:
        out *= d.value
    return out


In [7]:
input_tensor = tf.placeholder(tf.float32, [None, 
                                           N_EVENT_COLS, 
                                           N_EVENT_ROWS, 
                                           N_EVENT_SLICES, 
                                           2])
dropout_keep_prob = tf.placeholder(tf.float32)

conv1 = tf.nn.conv3d(input_tensor, init_weights([3,3,2,2,6]), 
                     strides=[1,1,1,1,1], padding='SAME')
conv1 = tf.nn.relu(tf.nn.bias_add(conv1, init_bias(n_output_channels(conv1))))
pool1 = tf.nn.max_pool3d(conv1, ksize=[1, 2, 2, 2, 1], 
                         strides=[1, 2, 2, 2, 1], padding="SAME")

conv2 = tf.nn.conv3d(pool1, 
                     init_weights([3,3,2, n_output_channels(pool1),6]), 
                     strides=[1, 1, 1, 1, 1], padding="SAME")
conv2 = tf.nn.relu(tf.nn.bias_add(conv2, init_bias(n_output_channels(conv2))))
pool2 = tf.nn.max_pool3d(conv1, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding="SAME")

fc3 = tf.reshape(pool2, [-1, n_outputs(pool2)])
fc3 = tf.add(tf.matmul(fc3, init_weights([n_outputs(pool2), 32])), init_bias(32))
fc3 = tf.nn.relu(fc3)
fc3 = tf.nn.dropout(fc3, dropout_keep_prob)

out = tf.add(tf.matmul(fc3, init_weights([n_outputs(fc3), 3])), init_bias(3))

y = tf.placeholder(tf.float32, [None, 3])

loss = tf.nn.l2_loss(tf.sub(out, y))
avg_loss = tf.reduce_mean(loss)
train_optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

init = tf.initialize_all_variables()

In [ ]:
TRAINING_STEPS = 10

sess = tf.Session()
sess.run(init)

for step in range(TRAINING_STEPS):
    for i in range(0, source.num_training - BATCH_SIZE, BATCH_SIZE):
        print(step, i)
        events = source.training()["events"][i:(i+BATCH_SIZE),:,...,:]
        labels = source.training()["labels"][i:(i+BATCH_SIZE),:]
        sess.run(train_optimizer, feed_dict={input_tensor: events,
                                             y: labels,
                                             dropout_keep_prob: 0.5})

    events_test = source.testing()["events"]
    labels_test = source.testing()["labels"]
    avg = sess.run(avg_loss, feed_dict={input_tensor: events_test,
                                        y: labels_test,
                                        dropout_keep_prob: 1.0})
    print("avg:", avg)
avg

0 0
0 5
0 10
0 15
0 20
0 25
0 30
0 35
0 40
0 45
0 50
0 55
0 60
0 65
0 70
0 75
0 80
0 85
0 90
0 95
0 100
0 105
0 110
0 115
0 120
0 125
0 130
0 135
0 140
0 145
0 150
0 155
0 160
0 165
0 170
0 175
0 180
0 185
0 190
0 195
0 200
0 205
0 210
0 215
0 220
0 225
0 230
0 235
0 240
0 245
